<img style="float:left;" src="../images/zcore.png" width="500" height="500" />

In [1]:
import bql
import bqplot as bqp
import pandas as pd

import ipywidgets as ipw
from ipywidgets import interact, interactive
from IPython.display import display

bq = bql.Service()

### Define Universe & Fields, Fetch Data

In [2]:
bindex_list = ['LG30TRUU Index', 'LF98TRUU Index', 'LUACTRUU Index', 'LGCPTRUU Index']

In [3]:
def runquery(bondindex):
    Members = bq.univ.members(bondindex)

    dates = bq.func.range('2019-12-31','2021-03-15')

    spread = bq.func.dropna(bq.data.spread(dates=dates))
    z_score = bq.func.z_score(spread)
    
    current_spread = bq.func.last(spread)
    current_z_score = bq.func.last(z_score)

    rank = bq.func.grouprank(current_z_score)  #derived data element
    Filtered_Members = bq.univ.filter(Members,rank<=200)  #filtered universe

    #addl description data
    long_comp_name = bq.data.name()
    fund_ticker = bq.func.value(bq.data.ID(), bq.univ.translatesymbols(instrumentidtype='Corp', targetidtype='FundamentalTicker'), mapby='lineage')
    fund_ticker_name = bq.func.value(bq.data.name(),bq.univ.translatesymbols(instrumentidtype='Corp', targetidtype='FundamentalTicker'), mapby='lineage' )
    sector = bq.data.classification_name()
    cusip= bq.data.id_cusip()
    rating = bq.data.bb_composite()

    fields = {'CUSIP':cusip, 'Rating':rating, 'Name': long_comp_name, 'Parent':fund_ticker, 'Parent_Name':fund_ticker_name, 'Sector':sector, 'Current Spread': current_spread, 'Current Z Score': current_z_score}


    request = bql.Request(Filtered_Members,fields)
    response = bq.execute(request)
    tmp = pd.DataFrame({r.name:r.df()[r.name] for r in response})
    return tmp

### Link Query to Dropdown

In [4]:
w= interactive(runquery,bondindex=ipw.Dropdown(options=bindex_list))
display(w)
#runquery('LF98TRUU Index')

interactive(children=(Dropdown(description='bondindex', options=('LG30TRUU Index', 'LF98TRUU Index', 'LUACTRUU…

### Submit Query, Tidy Results

In [5]:
tbl = w.result
tbl.sample(9)

,CUSIP,Rating,Name,Parent,Parent_Name,Sector,Current Spread,Current Z Score
ID,,,,,,,,
BM047535 Corp,00287YBX6,BBB,ABBV 3.2 11/21/29,ABBV US Equity,AbbVie Inc,Health Care,99.138870,1.750429
BN340908 Corp,501044DQ1,BBB,KR 1.7 01/15/31,KR US Equity,Kroger Co/The,Consumer Staples,84.206642,2.372190
BN352289 Corp,166756AX4,AA-,CVX 4.2 10/15/49,CVX US Equity,Chevron Corp,Energy,147.287140,1.447473
ZO479991 Corp,92939UAC0,BBB+,WEC 0.55 09/15/23,WEC US Equity,WEC Energy Group Inc,Utilities,31.497541,2.409060
BN972445 Corp,22822VAV3,BBB-,CCI 1.05 07/15/26,CCI US Equity,Crown Castle International Cor,Real Estate,76.530487,1.522811
BN849494 Corp,037833EC0,AA+,AAPL 1.2 02/08/28,AAPL US Equity,Apple Inc,Technology,44.345978,1.604264
ZO140988 Corp,29364GAM5,BBB,ETR 0.9 09/15/25,ETR US Equity,Entergy Corp,Utilities,61.053562,2.044622
BM747347 Corp,446413AS5,BBB-,HII 3.844 05/01/25,HII US Equity,Huntington Ingalls Industries,Industrials,73.037903,1.671551
BM390340 Corp,92343VFT6,BBB+,VZ 2.65 11/20/40,VZ US Equity,Verizon Communications Inc,Communications,139.537415,1.476685


### Visualize Data

In [6]:
from ipydatagrid import DataGrid

In [7]:
dg = DataGrid(dataframe=tbl)
dg

DataGrid(default_renderer=TextRenderer(), header_renderer=None)

In [8]:
dataframe = tbl.reset_index()

## Visualize with `bqviz` 

In [9]:
import bqviz as bqv

In [10]:
viz = bqv.InteractiveScatterPlot(df=tbl)

viz.show()

## Visualize with `bqapps`

In [11]:
import bqapps

In [12]:
bqapps.ScatterGrid(scatter_dataframe=tbl,
                  grid_dataframe=tbl,
                  scatter_color_column='Rating',
                  id_column='ID')

ScatterGrid(default_cell_renderer=TextRenderer(), id_column='ID', layout=Layout(height='550px', width='auto'),…

## Visualize with `plotly`

In [13]:
import plotly.express as px

In [14]:
tickerlist = sorted(tbl[tbl.Sector.notnull()].Sector.unique().tolist())

In [15]:
def issuerplot(name):
    fig_scat = px.scatter(tbl[tbl.Sector ==name].dropna(), x="Current Spread", y="Current Z Score", 
                      color='Parent_Name',
                      hover_data=['CUSIP', 'Rating'],
#                     facet_col='Parent_Name',
#                      facet_col_wrap=3,
                      title = name)
    fig_scat.show()

In [16]:
interact(issuerplot,name=ipw.Dropdown(options=tickerlist))

interactive(children=(Dropdown(description='name', options=('Communications', 'Consumer Discretionary', 'Consu…

<function __main__.issuerplot(name)>